In [1]:
#pip install -U pysal

In [2]:
import requests
import urllib.request
from zipfile import ZipFile
import json
import io
import zipfile
import geopandas as gpd
from shapely.geometry import shape  
import osr
import pandas as pd
import pysal as ps
import numpy as np
import requests
import geoplot
import geoplot.crs as gcrs
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# The following does not contain Alaska, Hawaii, and other islands.

### BG_2019

In [4]:
states= ['01','04','05','06','08','09','10','11','12','13','16']
col_names= ["AFFGEOID", "GEOID", "ALAND", "geometry"]
Geo_19= gpd.GeoDataFrame()
for i in states:
    url= 'https://www2.census.gov/geo/tiger/GENZ2019/shp/cb_2019_{}_bg_500k.zip'.format(i)
    BG_2019 = gpd.read_file(url)
    BG_2019.crs
    temp= gpd.GeoDataFrame(BG_2019)
    frame = [Geo_19, temp]
    Geo_19 = Geo_19.append(frame)

KeyboardInterrupt: 

In [ ]:
Geo_19.crs

In [ ]:
Geo_19.info()

In [ ]:
duplicate = Geo_19[Geo_19.duplicated(['GEOID'])]
duplicate

In [ ]:
Geo_19_clean= Geo_19.drop_duplicates(subset=['GEOID'])

In [ ]:
Geo_19_clean.info()

In [ ]:
#Geo_df_19= Merged_df_19.rename({'geometry': 'geometry19'}, axis=1)
#Geo_df_19.info()

In [ ]:
#API Census
states= ['01','04']
col_names= ["Name","Pop_19", "state", "county", "tract", "block group"]
df_API_19 = pd.DataFrame(columns=col_names)
for i in states:
    pop_url_str= 'https://api.census.gov/data/2019/acs/acs5?get=NAME,B01001_001E&for=block%20group:*&in=state:{}&in=county:*&in=tract:*'.format(i)
    r= requests.get(pop_url_str)
    temp= pd.DataFrame (columns=col_names, data=r.json()[1:])
    temp["Pop_19"]= temp["Pop_19"].astype(int)
    frame = [df_API_19, temp]
    df_API_19 = pd.concat(frame)

In [ ]:
df_API_19.info()

In [ ]:
df_API_19["Pop_19"]= df_API_19["Pop_19"].astype(int)

In [ ]:
df_API_19['GeoID19']= df_API_19['state']+ df_API_19['county']+ df_API_19['tract']+ df_API_19['block group']
df_API_19

In [ ]:
Join19= df_API_19.set_index('GeoID19').join(Geo_19_clean.set_index('GEOID'))

In [ ]:
Join19.info()

In [ ]:
sq_meter_per_square_mile = 2589988.1103
Join19['density19']=Join19['Pop_19']/(Join19['ALAND']/sq_meter_per_square_mile)

In [ ]:
Join19.info()

In [ ]:
Join19

In [ ]:
Join19.iloc[:, 0:].sort_values(by=['density19'], ascending=False).head(120)

### BG-2018

In [ ]:
states= ['01','04','05','06','08','09','10','11','12','13','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','44','45','46','47','48','49','50','51','53','54','55','56']
col_names= ["AFFGEOID", "GEOID", "ALAND"]
Merged_df_18 = pd.DataFrame(columns=col_names)
for i in states:
    url= 'https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_{}_bg_500k.zip'.format(i)
    BG_2018 = gpd.read_file(url)
    temp= pd.DataFrame (columns=col_names, data=BG_2018)
    temp["ALAND"]= temp["ALAND"].astype(int)
    frame = [Merged_df_18, temp]
    Merged_df_18 = pd.concat(frame)

In [ ]:
Merged_df_18["ALAND"]= Merged_df_18["ALAND"].astype(int)

In [ ]:
#Geo_df_18= Merged_df_18.rename({'geometry': 'geometry18'}, axis=1)
Merged_df_18

In [ ]:
#API Census
states= ['01','04','05','06','08','09','10','11','12','13','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','44','45','46','47','48','49','50','51','53','54','55','56']
col_names= ["Name","Pop_18", "state", "county", "tract", "block group"]
df_API_18 = pd.DataFrame(columns=col_names)
for i in states:
    pop_url_str= 'https://api.census.gov/data/2018/acs/acs5?get=NAME,B01001_001E&for=block%20group:*&in=state:{}&in=county:*&in=tract:*'.format(i)
    r= requests.get(pop_url_str)
    temp= pd.DataFrame (columns=col_names, data=r.json()[1:])
    temp["Pop_18"]= temp["Pop_18"].astype(int)
    frame = [df_API_18, temp]
    df_API_18 = pd.concat(frame)

In [ ]:
df_API_18["Pop_18"]= df_API_18["Pop_18"].astype(int)

In [ ]:
df_API_18['GeoID18']= df_API_18['state']+ df_API_18['county']+ df_API_18['tract']+ df_API_18['block group']

In [ ]:
df_18= df_API_18.drop(['state', 'county','tract','block group'], axis = 1)
df_18

In [ ]:
Join18= df_API_18.set_index('GeoID18').join(Merged_df_18.set_index('GEOID'))

In [ ]:
sq_meter_per_square_mile = 2589988.1103
Join18['density18']=Join18['Pop_18']/(Join18['ALAND']/sq_meter_per_square_mile)

In [ ]:
Join18

### BG-2017

In [ ]:
states= ['01','04','05','06','08','09','10','11','12','13','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','44','45','46','47','48','49','50','51','53','54','55','56']
col_names= ["AFFGEOID", "GEOID", "ALAND", "geometry"]
Merged_df_17 = pd.DataFrame(columns=col_names)
for i in states:
    url= 'https://www2.census.gov/geo/tiger/GENZ2017/shp/cb_2017_{}_bg_500k.zip'.format(i)
    BG_2017 = gpd.read_file(url)
    temp= pd.DataFrame (columns=col_names, data=BG_2017)
    temp["ALAND"]= temp["ALAND"].astype(int)
    frame = [Merged_df_17, temp]
    Merged_df_17 = pd.concat(frame)

In [ ]:
Merged_df_17["ALAND"]= Merged_df_17["ALAND"].astype(int)

In [ ]:
#Geo_df_17= Merged_df_17.rename({'geometry': 'geometry17'}, axis=1)

In [ ]:
#API Census
states= ['01','04','05','06','08','09','10','11','12','13','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','44','45','46','47','48','49','50','51','53','54','55','56']
col_names= ["Name","Pop_17", "state", "county", "tract", "block group"]
df_API_17 = pd.DataFrame(columns=col_names)
for i in states:
    pop_url_str= 'https://api.census.gov/data/2017/acs/acs5?get=NAME,B01001_001E&for=block%20group:*&in=state:{}&in=county:*&in=tract:*'.format(i)
    r= requests.get(pop_url_str)
    temp= pd.DataFrame (columns=col_names, data=r.json()[1:])
    temp["Pop_17"]= temp["Pop_17"].astype(int)
    frame = [df_API_17, temp]
    df_API_17 = pd.concat(frame)

In [ ]:
df_API_17["Pop_17"]= df_API_17["Pop_17"].astype(int)

In [ ]:
df_API_17['GeoID17']= df_API_17['state']+ df_API_17['county']+ df_API_17['tract']+ df_API_17['block group']

In [ ]:
df_17= df_API_17.drop(['state', 'county','tract','block group'], axis = 1)

In [ ]:
Join17= df_17.set_index('GeoID17').join(Merged_df_17.set_index('GEOID'))

In [ ]:
sq_meter_per_square_mile = 2589988.1103
Join17['density17']=Join17['Pop_17']/(Join17['ALAND']/sq_meter_per_square_mile)
Join17

### BG-2016

In [ ]:
states= ['01','04','05','06','08','09','10','11','12','13','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','44','45','46','47','48','49','50','51','53','54','55','56']
col_names= ["AFFGEOID", "GEOID", "ALAND", "geometry"]
Merged_df_16 = pd.DataFrame(columns=col_names)
for i in states:
    url= 'https://www2.census.gov/geo/tiger/GENZ2016/shp/cb_2016_{}_bg_500k.zip'.format(i)
    BG_2016 = gpd.read_file(url)
    temp= pd.DataFrame (columns=col_names, data=BG_2016)
    temp["ALAND"]= temp["ALAND"].astype(int)
    frame = [Merged_df_16, temp]
    Merged_df_16 = pd.concat(frame)

In [ ]:
Merged_df_16["ALAND"]= Merged_df_16["ALAND"].astype(int)

In [ ]:
#Geo_df_16= Merged_df_16.rename({'geometry': 'geometry16'}, axis=1)

In [ ]:
#API Census
states= ['01','04','05','06','08','09','10','11','12','13','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','44','45','46','47','48','49','50','51','53','54','55','56']
col_names= ["Name","Pop_16", "state", "county", "tract", "block group"]
df_API_16 = pd.DataFrame(columns=col_names)
for i in states:
    pop_url_str= 'https://api.census.gov/data/2016/acs/acs5?get=NAME,B01001_001E&for=block%20group:*&in=state:{}&in=county:*&in=tract:*'.format(i)
    r= requests.get(pop_url_str)
    temp= pd.DataFrame (columns=col_names, data=r.json()[1:])
    temp["Pop_16"]= temp["Pop_16"].astype(int)
    frame = [df_API_16, temp]
    df_API_16 = pd.concat(frame)

In [ ]:
df_API_16["Pop_16"]= df_API_16["Pop_16"].astype(int)

In [ ]:
df_API_16['GeoID16']= df_API_16['state']+ df_API_16['county']+ df_API_16['tract']+ df_API_16['block group']

In [ ]:
df_16= df_API_16.drop(['state', 'county','tract','block group'], axis = 1)

In [ ]:
Join16= df_16.set_index('GeoID16').join(Merged_df_16.set_index('GEOID'))

In [ ]:
sq_meter_per_square_mile = 2589988.1103
Join16['density16']=Join16['Pop_16']/(Join16['ALAND']/sq_meter_per_square_mile)

In [ ]:
Join16

### BG-2015

In [ ]:
states= ['01','04','05','06','08','09','10','11','12','13','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','44','45','46','47','48','49','50','51','53','54','55','56']
col_names= ["AFFGEOID", "GEOID", "ALAND"]
Merged_df_15 = pd.DataFrame(columns=col_names)
for i in states:
    url= 'https://www2.census.gov/geo/tiger/GENZ2015/shp/cb_2015_{}_bg_500k.zip'.format(i)
    BG_2015 = gpd.read_file(url)
    temp= pd.DataFrame (columns=col_names, data=BG_2015)
    temp["ALAND"]= temp["ALAND"].astype(int)
    frame = [Merged_df_15, temp]
    Merged_df_15 = pd.concat(frame)

In [ ]:
Merged_df_15["ALAND"]= Merged_df_15["ALAND"].astype(int)

In [ ]:
#Geo_df_15= Merged_df_15.rename({'geometry': 'geometry15'}, axis=1)

In [ ]:
#API Census
states= ['01','04','05','06','08','09','10','11','12','13','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','44','45','46','47','48','49','50','51','53','54','55','56']
col_names= ["Name","Pop_15", "state", "county", "tract", "block group"]
df_API_15 = pd.DataFrame(columns=col_names)
for i in states:
    pop_url_str= 'https://api.census.gov/data/2015/acs/acs5?get=NAME,B01001_001E&for=block%20group:*&in=state:{}&in=county:*&in=tract:*'.format(i)
    r= requests.get(pop_url_str)
    temp= pd.DataFrame (columns=col_names, data=r.json()[1:])
    temp["Pop_15"]= temp["Pop_15"].astype(int)
    frame = [df_API_15, temp]
    df_API_15 = pd.concat(frame)

In [ ]:
df_API_15["Pop_15"]= df_API_15["Pop_15"].astype(int)

In [ ]:
df_API_15['GeoID15']= df_API_15['state']+ df_API_15['county']+ df_API_15['tract']+ df_API_15['block group']

In [ ]:
df_15= df_API_15.drop(['state', 'county','tract','block group'], axis = 1)

In [ ]:
Join15= df_15.set_index('GeoID15').join(Merged_df_15.set_index('GEOID'))

In [ ]:
sq_meter_per_square_mile = 2589988.1103
Join15['density15']=Join15['Pop_15']/(Join15['ALAND']/sq_meter_per_square_mile)

In [ ]:
Join15

### BG-2014

In [ ]:
states= ['01','04','05','06','08','09','10','11','12','13','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','44','45','46','47','48','49','50','51','53','54','55','56']
col_names= ["AFFGEOID", "GEOID", "ALAND"]
Merged_df_14 = pd.DataFrame(columns=col_names)
for i in states:
    url= 'https://www2.census.gov/geo/tiger/GENZ2014/shp/cb_2014_{}_bg_500k.zip'.format(i)
    BG_2014 = gpd.read_file(url)
    temp= pd.DataFrame (columns=col_names, data=BG_2014)
    temp["ALAND"]= temp["ALAND"].astype(int)
    frame = [Merged_df_14, temp]
    Merged_df_14 = pd.concat(frame)

In [ ]:
Merged_df_14["ALAND"]= Merged_df_14["ALAND"].astype(int)

In [ ]:
#Geo_df_14= Merged_df_14.rename({'geometry': 'geometry14'}, axis=1)

In [ ]:
#API Census
states= ['01','04','05','06','08','09','10','11','12','13','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','44','45','46','47','48','49','50','51','53','54','55','56']
col_names= ["Name","Pop_14", "state", "county", "tract", "block group"]
df_API_14 = pd.DataFrame(columns=col_names)
for i in states:
    pop_url_str= 'https://api.census.gov/data/2014/acs/acs5?get=NAME,B01001_001E&for=block%20group:*&in=state:{}&in=county:*&in=tract:*'.format(i)
    r= requests.get(pop_url_str)
    temp= pd.DataFrame (columns=col_names, data=r.json()[1:])
    temp["Pop_14"]= temp["Pop_14"].astype(int)
    frame = [df_API_14, temp]
    df_API_14 = pd.concat(frame)

In [ ]:
df_API_14["Pop_14"]= df_API_14["Pop_14"].astype(int)

In [ ]:
df_API_14['GeoID14']= df_API_14['state']+ df_API_14['county']+ df_API_14['tract']+ df_API_14['block group']

In [ ]:
df_14= df_API_14.drop(['state', 'county','tract','block group'], axis = 1)

In [ ]:
Join14= df_14.set_index('GeoID14').join(Merged_df_14.set_index('GEOID'))

In [ ]:
sq_meter_per_square_mile = 2589988.1103
Join14['density14']=Join14['Pop_14']/(Join14['ALAND']/sq_meter_per_square_mile)

In [ ]:
Join14

### BG-2013

In [ ]:
states= ['01','04','05','06','08','09','10','11','12','13','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','44','45','46','47','48','49','50','51','53','54','55','56']
col_names= ["AFFGEOID", "GEOID", "ALAND"]
Merged_df_13 = pd.DataFrame(columns=col_names)
for i in states:
    url= 'https://www2.census.gov/geo/tiger/GENZ2013/cb_2013_{}_bg_500k.zip'.format(i)
    BG_2013 = gpd.read_file(url)
    temp= pd.DataFrame (columns=col_names, data=BG_2013)
    temp["ALAND"]= temp["ALAND"].astype(int)
    frame = [Merged_df_13, temp]
    Merged_df_13 = pd.concat(frame)

In [ ]:
Merged_df_13["ALAND"]= Merged_df_13["ALAND"].astype(int)

In [ ]:
#Geo_df_13= Merged_df_13.rename({'geometry': 'geometry13'}, axis=1)

In [ ]:
#API Census
states= ['01','04','05','06','08','09','10','11','12','13','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','44','45','46','47','48','49','50','51','53','54','55','56']
col_names= ["Name","Pop_13", "state", "county", "tract", "block group"]
df_API_13 = pd.DataFrame(columns=col_names)
for i in states:
    pop_url_str= 'https://api.census.gov/data/2013/acs/acs5?get=NAME,B01001_001E&for=block%20group:*&in=state:{}&in=county:*&in=tract:*'.format(i)
    r= requests.get(pop_url_str)
    temp= pd.DataFrame (columns=col_names, data=r.json()[1:])
    temp["Pop_13"]= temp["Pop_13"].astype(int)
    frame = [df_API_13, temp]
    df_API_13 = pd.concat(frame)

In [ ]:
df_API_13["Pop_13"]= df_API_13["Pop_13"].astype(int)

In [ ]:
df_API_13['GeoID13']= df_API_13['state']+ df_API_13['county']+ df_API_13['tract']+ df_API_13['block group']

In [ ]:
df_13= df_API_13.drop(['state', 'county','tract','block group'], axis = 1)

In [ ]:
Join13= df_13.set_index('GeoID13').join(Merged_df_13.set_index('GEOID'))

In [ ]:
sq_meter_per_square_mile = 2589988.1103
Join13['density13']=Join13['Pop_13']/(Join13['ALAND']/sq_meter_per_square_mile)
Join13

### BG-2012

In [ ]:
states= ['01','04','05','06','08','09','10','11','12','13','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','44','45','46','47','48','49','50','51','53','54','55','56']
col_names= ["AFFGEOID", "GEOID", "ALAND", "geometry"]
Merged_df_12 = pd.DataFrame(columns=col_names)
for i in states:
    url= 'https://www2.census.gov/geo/tiger/TIGER2012/BG/tl_2012_{}_bg.zip'.format(i)
    BG_2012 = gpd.read_file(url)
    temp= pd.DataFrame (columns=col_names, data=BG_2012)
    temp["ALAND"]= temp["ALAND"].astype(int)
    frame = [Merged_df_12, temp]
    Merged_df_12 = pd.concat(frame)

In [ ]:
Merged_df_12["ALAND"]= Merged_df_12["ALAND"].astype(int)

In [ ]:
Geo_df_12= Merged_df_12.rename({'geometry': 'geometry12'}, axis=1)

### BG-2011

In [ ]:
states= ['01','04','05','06','08','09','10','11','12','13','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','44','45','46','47','48','49','50','51','53','54','55','56']
col_names= ["AFFGEOID", "GEOID", "ALAND", "geometry"]
Merged_df_11 = pd.DataFrame(columns=col_names)
for i in states:
    url= 'https://www2.census.gov/geo/tiger/TIGER2011/BG/tl_2011_{}_bg.zip'.format(i)
    BG_2011 = gpd.read_file(url)
    temp= pd.DataFrame (columns=col_names, data=BG_2011)
    temp["ALAND"]= temp["ALAND"].astype(int)
    frame = [Merged_df_11, temp]
    Merged_df_11 = pd.concat(frame)

In [ ]:
Merged_df_11["ALAND"]= Merged_df_11["ALAND"].astype(int)

In [ ]:
Geo_df_11= Merged_df_11.rename({'geometry': 'geometry11'}, axis=1)

## BG-2010

In [ ]:
states= ['01','04','05','06','08','09','10','11','12','13','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','44','45','46','47','48','49','50','51','53','54','55','56']
col_names= ["AFFGEOID", "GEOID", "ALAND", "geometry"]
Merged_df_10 = pd.DataFrame(columns=col_names)
for i in states:
    url= 'https://www2.census.gov/geo/tiger/TIGER2010/BG/2010/tl_2010_{}_bg10.zip'.format(i)
    BG_2010 = gpd.read_file(url)
    temp= pd.DataFrame (columns=col_names, data=BG_2011)
    temp["ALAND"]= temp["ALAND"].astype(int)
    frame = [Merged_df_10, temp]
    Merged_df_10 = pd.concat(frame)

In [ ]:
Merged_df_10["ALAND"]= Merged_df_10["ALAND"].astype(int)

In [ ]:
Geo_df_10= Merged_df_10.rename({'geometry': 'geometry10'}, axis=1)

In [ ]:
df_Pop_Merge= Join19.merge(Join18, on='Name').merge(Join17, on='Name').merge(Join16, on='Name').merge(Join15, on='Name').merge(Join14, on='Name').merge(Join13, on='Name')

In [ ]:
df_Pop_Merge.info()

In [ ]:
df_Pop_Merge= df_Pop_Merge.drop(['AFFGEOID_x', 'AFFGEOID_y', 'geometry_x', 'geometry_y'], axis = 1)
df_Pop_Merge.info()

In [ ]:
df_geo= df_Pop_Merge.rename({'geometry19': 'geometry'}, axis=1)

In [ ]:
#calculate population change
df_geo['%PopChange']=((df_geo['Pop_19']-df_geo['Pop_13'])/df_geo['Pop_13'])*100

In [ ]:
df_geo

In [ ]:
df_clean= df_geo.dropna(subset=['geometry', ])
df_clean

### Visualization

In [ ]:
def rd2(x):
        return round (x,2)
    
minimum, maximum= Join19["density19"].quantile([0.05, 0.95]).apply(rd2)
mean = round(Join19["density19"].mean(), 2)


print(f"minimum: {minimum}", f"maximum: {maximum}", f"Mean: {mean}", sep="\n\n")  

In [ ]:
import branca

colormap = branca.colormap.LinearColormap(
    colors=["#f2f0f7", "#cbc9e2", "#9e9ac8", "#756bb1", "#54278f"],
    index=Join19["density19"].quantile([0.2, 0.4, 0.6, 0.8]),
    vmin=minimum,
    vmax=maximum,
)

colormap.caption = "Population Change Percentage"

colormap

In [ ]:
import folium
from folium.plugins import Search


m = folium.Map(location=[38, -97], zoom_start=4)

In [ ]:
df_clean_19= df_map.dropna(subset=['geometry','density19'])
df_clean_19.info()

In [ ]:
#df_map= gpd.GeoDataFrame(Join19, geometry='geometry')

In [ ]:
#df_clean_19['geometry19'] = gpd.GeoSeries.from_wkt(df_clean_19['geometry19'])
#my_geo_df = gpd.GeoDataFrame(df_clean_19, geometry='geometry19')

In [ ]:
#from shapely import wkt

#df_clean_19['geometry19'] = df_clean_19['geometry19'].apply(wkt.loads)
#my_geo_df = gpd.GeoDataFrame(df_clean_19, geometry='geometry19')

In [ ]:
print(type(Join19))

In [ ]:
df_map= gpd.GeoDataFrame(Join19, geometry='geometry')

In [ ]:
# cx coordinate based indexing
# http://geopandas.org/indexing.html
df_clean_19 = df_clean_19.cx[-120:-65, 0:80]

num_colors = 40
vmin=0
vmax=300

# https://stackoverflow.com/questions/39215202/matplotlib-geopandas-plot-multiple-layers-control-figsize
figsize = (20, 20)
fig, ax = plt.subplots(1, 1, figsize=figsize)
ax.set_title('Population density [people/sq mile]')
df_clean_19.plot(column='density19', cmap='OrRd', ax=ax, k=num_colors, legend=True, vmin=vmin, vmax=vmax, legend_kwds={'label': "Population Density", 'orientation': "horizontal"})

In [ ]:
def style_function(x):
    return {
        "fillColor": colormap(x["properties"]["density19"]),
        "color": "black",
        "weight": 2,
        "fillOpacity": 0.5,
    }


stategeo = folium.GeoJson(
    df_clean_19,
    name="US Block Goups",
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=["Name", "density19"], aliases=["Name", "Density"], localize=True
    ),
).add_to(m)



folium.LayerControl().add_to(m)
colormap.add_to(m)

m

In [ ]:
#basemap (used state cap. for the lat/long)
P_Density = folium.Map(location = [40.29794048212508, -100.8402113505973], zoom_start = 7, control_scale = True)

#create a choropleth layer of Global Rare species density:
folium.Choropleth(
    geo_data = df_clean_19,
    name = 'US Block Goups',
    data = df_clean_19,
    columns = ['Name', 'density19'],
    key_on = 'feature.id',
    fill_color = 'GnBu',
    fill_opacity = 0.8,
    line_opacity = 0.2,
    line_color = 'black',
    line_weight = 1,
    highlight = True,
    overlay = True,
    show = True,
    smooth_factor = 1.0,
    legend_name = '%Population Density Change').add_to(P_Density)







# Add hover functionality.
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
NIL = folium.features.GeoJson(
    data = df_clean_19,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['density19'],
        aliases=['density19'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
P_Density.add_child(NIL)
P_Density.keep_in_front(NIL)


#add counties polygons
folium.GeoJson(P_Density, style_function = lambda x: {
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0
    }, name = 'density19').add_to(P_Density)

#enable layer control
folium.LayerControl().add_to(P_Density)

#show map will this work better?
P_Density